In [27]:
import cv2
import time
import json
import random
from PIL import Image
import numpy as np
from mss import mss
import serial, time
import binascii

In [48]:
cap = cv2.VideoCapture(0)
sct = mss()

In [49]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
src_dev = 0
w, h = 64, 64
monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}

In [51]:
while True:
    while True:
        rline = arduino.readline().strip()
        if len(rline) > 0:
            print(rline)
        else:
            break
        
    if src_dev == 0:
        ret, frame = cap.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        rgb = np.array(sct.grab(monitor))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

    obuff=cv2.resize(rgb, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    #print(obuff)
    #break
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    #print(binascii.hexlify(nobuff))

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64

        ln = arduino.write(sbuff)
        arduino.flush()
        print('%s, %s' % (ln, binascii.hexlify(sbuff)))
        time.sleep(0.1)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(1)
    #time.sleep(2)
    #break


65, b'00fff7fffffff3fffffffffffffff3fffffcf7ffffffd3fffffcffffffffd3fffff0ffffffffd3ffffe0ffffffff03ffffc0ffffffff81ffff80ffffffff00ffff'
65, b'0180fcffffff00feff00f7ffff7f00feff00efffff3f00fcff00ffffff3f1cfeff00ffffff3f1cfeff00feffff0ffcffff00feffff0ff8ffff00fcffff0ff0ffff'
65, b'0200f8ffff03e0ffff00e0ffff01f0ffff0080ffff00e0ffff0000fe7f00c0ffff0000fe3f00c0ffff0000eebf0080ffff0000fc7f0780ffff0000f8ff0380ffff'
65, b'030080ff7f0000ffff00f8ff1600c0ffff0018e01300c0ffff000000080080ffff0000c01f0000feff0000807f0000fcff0000e0110000f8ff000040110000e0ff'
65, b'040000c0110000c0ff0000b0190000c0ff0000b0000000c0ff000020000080c0ff0000e4000000e0ff000000000000f1ff000000000000f1ff000000000000f1ff'
65, b'05000020000000fbff0000a8000000f9ff00005e000000f1ff00001c00000083ff00000600000082ff00001c00000080ff00003e00000080ff00001f00000080ff'
65, b'0600003c000000c0ff00001f000000c0df00000a000000e0ff00400f000000e0ff00001e000000e0ff00c01f0000f0e07f00c03d0000e0e3ff00800f0000e0ffff'
65, b'0700800f0000ccffdf00c0170000

65, b'040000000000ecffff0000000000ecffff0000000000eeffff0000000010cfffff0000000010ceffff0000000030cfffff0000000030cfffff0000000030cfffff'
65, b'050000000010ccffff0000000020c8ffff0000000020c8ffff0000000000c8ffff0000000000c0ffff0000000000c0ffff0000000000c1ffff0000000000c2ff7f'
65, b'060000000000c0ff7f000000000080ff57000000000080ff3f000000000080ff5f000000000080ff3f000000000080ff07000000000080ff11000000003080ff01'
65, b'0700000000e082ff000000000090077f000000000010077e0000000000200648000000000020040000000000000004000000000000000000000000000000000000'
65, b'0000e0ffff0fc0ffff0000ffff01c0ffff0000fdff00c0ffff0000fc5f00c0ffff0000d40f00c0ffff0000e00f00c0ffff0000c00700c0ffff0000000700c0ffff'
65, b'010004000000e0ffff0000000000e1ffff0008000080e1ffff001c000080e3ffff003c0000c0e3ffff00180000c0e7ffff002c0000c0e7ffff00040000c0efffff'
65, b'0200000000c0ffffff00000000c0ffffff0000000080ffffff0000000080ffffff0000000080ffffff0000000080ffffff0000000080ffffff0000000080ffffff'
65, b'030000000080ffffff0000000000

65, b'0200f87f80feffffff00e07f00ffffffff00c07f00ffffffff00c07f00ffffffff00807d00feffffff00007e00feffffff00007e00fcffffff00007e00fcffffff'
65, b'0300807f00feffffff00f87f00feffffff00783400feffffff00000000feffffff00006000feffffff00004000fcffffff00007000f8ffffff0000700080ffffff'
65, b'040000700000ffffff0000600000ffffff0000a00040ffffff0000f80080ffffff00007800c0ffffff00006800c1ffffff00001800c1ffffff00008800c1ffffff'
65, b'0500007800c7ffffff0000fc00e7ffffff0000fe00c7ffffff00007f0003ecffff00007e0007e8ffff0000ff0007e8ffff00007e0007e8ffff00803f0046e0ffff'
65, b'0600803f0080e0ffff00c03f0080e0ffff00c01e0080e4ffff00001f0080c5ffff00e0060080c5ffff00e0010080d7ffff00d1000080c5ffff0060000000c5ffff'
65, b'0700400000828dffff004000009e8dffff00200000fe1dffff00000080ff1fffff00080080ff3fe9fb00000080ff3fc0f380000080df3fc0f300000000ff3f80f3'
65, b'00fff7fffffffffffffffffffffffffffffeffffff0ffffffff8ffffff07fefffffcffffff03fcfffff0ffffff01fcfffff0ffffff00fcffffe0ffffff00fbffff'
65, b'01c0feff7f00f6ffff00fbff3f00

65, b'0600feff010000e4ff00feff010000e4ff00fe3f000000f4ff00fe3f000030f4ff00ff3f00007efcff00ff1f0000fcfcff00ff1f0000f8fdff00ff1f0000f8ffff'
65, b'0700ff0700c0fbffff00ff070000feffff00ff0b0000feffff00ff030000fcffff00ff010000f0effb80ff050020f0eff180ff040000f18ff180ff040000e10ff1'


KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows()

In [ ]:
#w, h = 200, 200
#monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
#img = Image.frombytes('RGB', (w,h), sct.grab(monitor).rgb)
#cv2.imshow('frame', np.array(img))
cv2.imshow('frame', np.array(sct.grab(monitor)))
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
cap.release()

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
buff=bytearray(65)

In [ ]:
arduino.write(b'\x04'+buff)

In [ ]:
for i in range(65):
    buff[i] = i

In [ ]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


In [13]:
while True:
    print(arduino.readline())

b'\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x80\x00\x80\x80\x80\x80\x80\x00\x80\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x00\x80\x80\x00\x00\x80\x00\x00\x00\x80\x80\x00\x80\x00\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x00\x00\x00\x00\x00\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x80\x80\x80\x80\x00\x80\x00\x00\x80\x80\x80\x00\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x00\x80\x00\x80\x80\x80\x80\x00\x80\x00\x00\x80\x00\x00\x00\x80\x80\x80\x80\x80\x80\x80\x00\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x00\x00\x80\x00\x00\x00\x00\x00\x80\x00\x80\x00\x80\x80\x00\x80\x80\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x80\x80\x00\x00\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x00\x80\x80\x

KeyboardInterrupt: 